
# Yandex.Afisha Marketing Analytics

## Step 0. Project Description

This project is part of the Business Analytics course.  
I act as an intern in the analytical department of **Yandex.Afisha**.  
The goal is to help optimize **marketing expenses** using data from June 2017 to May 2018.

**Available data:**
- `visits_log_us.csv` — website visit logs  
- `orders_log_us.csv` — orders data  
- `costs_us.csv` — marketing expenses  

**Main questions to answer:**
1. How do users interact with the product? (DAU/WAU/MAU, sessions, retention, devices)
2. When and how do they buy? (conversion delay, AOV, LTV, devices)
3. How effective is marketing? (spend, CAC, ROI, payback, devices)

**Tools:** Python, Pandas, NumPy, Matplotlib, Seaborn, Plotly

**Format:** Jupyter Notebook with tidy code, comments, and markdown explanations.  

At the end, I will provide **recommendations for the marketing team** on how to distribute advertising budgets more effectively.

## Project Roadmap

**Steps:**

1. Data preparation — load datasets, fix types.
2. Product metrics — DAU/WAU/MAU, sessions, retention, session length by device.
3. Sales metrics — conversion delay, orders per user, AOV, LTV, AOV by device.
4. Marketing metrics — spend, CAC by source, ROI by cohorts, spend by device.
5. Conclusion — summary and marketing recommendations.

**Deliverables:**

1. Clean Jupyter Notebook with tidy code and comments.
2. Key plots for metrics by source and device.
3. Final recommendations for budget allocation.

## Step 1. Data Preparation

### 1.1. Imports

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.io as pio
import seaborn as sns
import matplotlib.dates as mdates
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
from IPython.display import display
import plotly.graph_objects as go
from typing import Optional

In [ ]:
# Set one style for graphs and charts
pio.templates.default = "plotly_white"

pio.templates["plotly_white"].layout.update(
    width=800,
    height=420,
    margin=dict(l=60, r=20, t=60, b=60),
    yaxis=dict(tickformat=",")
);

### 1.2. Utility functions (loader • cleaning • quick EDA)

In [ ]:
def to_snake(df: pd.DataFrame) -> pd.DataFrame:
    """Convert column names to snake_case."""
    out = df.copy()
    out.columns = (out.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(' ', '_'))
    return out


def explore_data(df: pd.DataFrame, name: str, n: int = 5) -> None:
    """Quick, repeatble EDA for any dataframe (shape, dtypes, nulls, duplicates, head/tail)."""
    print(f"\n{' Exploring' + name + ' ':=^80}")
    print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} cols\n")
    print("Dtypes:")
    display(df.dtypes.to_frame('dtype'))

    nulls = df.isna().sum()
    if nulls.sum() > 0:
        print("\nMissing values:")
        display(nulls[nulls > 0].sort_values(ascending=False).to_frame('n_missing'))
    else:
        print("\nMissing values: none")

    print(f"\nDuplicate rows: {df.duplicated().sum():,}")

    print(f"\nHead({n}):")
    display(df.head(n))
    print(f"\nTail ({n}):")
    display(df.tail(n))

In [ ]:
# Loaders with types
def load_visits(path: str) -> pd.DataFrame:
    """Read Visits with datetime parsing and device as category; then make snake_case."""
    df = pd.read_csv(path, parse_dates=['Start Ts', 'End Ts'], dtype={'Device': 'category'})
    return to_snake(df)

def load_orders(path: str) -> pd.DataFrame:
    """Read Orders with datetime parsing; then make snake_case."""
    df = pd.read_csv(path, parse_dates=['Buy Ts'])
    return to_snake(df) 

def load_costs(path: str) -> pd.DataFrame:
    """Read Costs with datetime parsing; already snake_case in source, keep consistent"""
    df = pd.read_csv(path, parse_dates=['dt'])
    return to_snake(df)

In [ ]:
# Cleaning / validation
def clean_visits(df: pd.DataFrame) -> pd.DataFrame:
    """Compute session duration (min) and drop rows with negetive duration (data anomalies)."""
    out = df.copy()
    out['session_duration_min'] = (out['end_ts'] - out['start_ts']).dt.total_seconds() / 60
    neg = (out['session_duration_min'] < 0).sum()
    if neg > 0:
        print(f"removed {neg} rows with negative session durations.")
        out = out[out['session_duration_min'] >= 0]
    return out

def validate_ranges(visits: pd.DataFrame, orders: pd.DataFrame, costs: pd.DataFrame) -> None:
    """Simple checks: date windows and negative values in key numeric fields"""
    print("\n=== Date ranges (min -> max) ===")
    print("Visits :", visits['start_ts'].min(), "->", visits['end_ts'].max())
    print("Orders :", orders['buy_ts'].min(),   "->", orders['buy_ts'].max())
    print("Costs  :", costs['dt'].min(),        "->", costs['dt'].max())

    print("\nNegatives check:")
    print("Orders with negative revenue:", (orders['revenue'] < 0).sum())
    print("Costs  with negative values :", (costs['costs'] < 0).sum())

### 1.3. Load datasets

In [ ]:
# Use the loaders
visits = load_visits('visits_log_us.csv')
orders = load_orders('orders_log_us.csv')
costs  = load_costs('costs_us.csv')

# Quick EDA per table
explore_data(visits, 'Visits')
explore_data(orders, 'Orders')
explore_data(costs,  'Costs')

### 1.4. Basic data cleaning & validation

In [ ]:
# Duplicates 
print('Duplicate rows:')
print('  visits:', visits.duplicated().sum())
print('  orders:', orders.duplicated().sum())
print('  costs :', costs.duplicated().sum())

# Clean Visits: remove negative session durations (if any)
visits = clean_visits(visits)

# Validate date ranges and negative numeric values
validate_ranges(visits, orders, costs)

### Summary

- Column names converted to `snake_case`  
- Datetime and numeric types parsed correctly  
- No duplicates or missing values found  
- Negative session durations removed (2 rows dropped)  
- Date ranges across visits, orders, and costs align with the analysis window (Jun 2017 – May 2018)  

The data is clean and ready for further analysis

## Step 2. Exploring Product Metrics (Usage)

In this section, we will answer the following key questions: 


- How many people use the product every day, week, and month (DAU / WAU / MAU)?
- How many sessions occur per day?
- What is the average session length?
- What is the user retention rate?
- How does session length differ by device?

### 2.1. How many people use the product every day, week, and month (DAU / WAU / MAU)

In [ ]:
# --- Activity metrics ---

def get_activity(visits: pd.DataFrame):
    """Return DAU, WAU, MAU as pandas Series."""
    v = visits.copy()
    v['date']  = v['start_ts'].dt.date
    v['week']  = v['start_ts'].dt.to_period('W')
    v['month'] = v['start_ts'].dt.to_period('M')
    
    dau = v.groupby('date')['uid'].nunique()
    wau = v.groupby('week')['uid'].nunique()
    mau = v.groupby('month')['uid'].nunique()
    
    return dau, wau, mau


# Compute metrics
dau, wau, mau = get_activity(visits)

print(f"DAU — Average: {dau.mean():.0f}, Median: {dau.median():.0f}")
print(f"WAU — Average: {wau.mean():.0f}, Median: {wau.median():.0f}")
print(f"MAU — Average: {mau.mean():.0f}, Median: {mau.median():.0f}")


# --- Visualization (DAU/WAU/MAU) ---

# 1. Prepare x-axes
x_dau = pd.to_datetime(dau.index)
x_wau = wau.index.to_timestamp()
x_mau = mau.index.to_timestamp()

# 2. Create subplots
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.07,
    subplot_titles=[
        "DAU — Daily Active Users",
        "WAU — Weekly Active Users",
        "MAU — Monthly Active Users"
    ]
)

# 3. Add traces
fig.add_trace(go.Scatter(x=x_dau, y=dau, mode="lines", line=dict(color="royalblue")), row=1, col=1)
fig.add_trace(go.Bar(x=x_wau, y=wau, marker_color="royalblue"), row=2, col=1)
fig.add_trace(go.Scatter(x=x_mau, y=mau, mode="lines+markers", line=dict(color="royalblue")), row=3, col=1)

# 4. Layout — keep only unique settings
fig.update_layout(
    title="DAU, WAU, MAU Metrics",
    height=750,       # custom height for this figure only
    showlegend=False
)

# 5. Axis labels
fig.update_yaxes(title_text="Users", row=1, col=1)
fig.update_yaxes(title_text="Users", row=2, col=1)
fig.update_yaxes(title_text="Users", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)

fig.show()

**Summary:**

- The DAU chart turned out too dense to be informative, so we rely on aggregated metrics instead.  
Both mean and median values are close to each other, which indicates no strong outliers.  
- WAU grew steadily from summer to autumn 2017, peaking in November before showing fluctuations and a gradual decline in spring 2018. The November spike should be further investigated to understand the drivers of user growth.  
- MAU followed a similar pattern: steady growth in 2017 with a peak in November–December, followed by a decline in 2018. This correlates with the weekly chart and may reflect seasonality. It is worth analyzing the reasons behind the decline after the peak period.

### 2.2. Number of sessions per day

We measure daily load as the total number of sessions per calendar day.  
To reduce day-to-day noise, we also show a 7-day rolling average and the median as a reference line.

In [ ]:
# Daily Sessions Analysis

# Prepare the date column
visits['date'] = visits['start_ts'].dt.date

# Count the number of sessions per day
sessions_per_day = (
    visits['date']
    .value_counts()
    .sort_index()
)

# Compute 7-day rolling mean (smoothing)
spd_smoothed = sessions_per_day.rolling(7).mean()

# Median daily sessions (for reference line)
median_spd = sessions_per_day.median()

# Print numeric summary
print(f"Median number of sessions per day: {median_spd:,.0f}")

# Plot
fig = px.line(
    spd_smoothed,
    labels={'index': 'Date', 'value': 'Sessions'},
    title='Number of Sessions per Day (7-Day Rolling Average)'
)

# Style line
fig.update_traces(line=dict(color='royalblue'))

# Add horizontal median line
fig.add_hline(
    y=median_spd,
    line_dash='dash',
    line_color='red',
    annotation_text='Median',
    annotation_position='top left'
)

fig.show()

**Summary**

- Daily sessions show clear fluctuations with prominent peaks around Nov 2017 and early 2018.  
- The median daily level is ~1,003 sessions (red dashed line), which serves as a stable baseline.  
- The 7-day smoothing helps reveal the overall trend without day-to-day noise.

### 2.3. Exploring Session Duration

In [ ]:
# Calculate session duration (in minutes)
session_durations = (visits['end_ts'] - visits['start_ts']) / pd.Timedelta(minutes=1)
session_durations = session_durations.astype(int)

# Compute summary statistics
mean_duration = session_durations.mean()
median_duration = session_durations.median()
min_duration = session_durations.min()
max_duration = session_durations.max()

# Print stats
print("Session Duration Summary (minutes):")
print(f"Average: {mean_duration:.0f}")
print(f"Median:  {median_duration:.0f}")
print(f"Minimum: {min_duration:.0f}")
print(f"Maximum: {max_duration:.0f}")

# Plot histogram
fig = px.histogram(
    session_durations,
    nbins=200,
    range_x=[-1, 120], 
    title='Distribution of Session Durations',
    labels={'value': 'Session duration (minutes)', 'count': 'Number of sessions'}
)


fig.update_traces(marker_color='royalblue', opacity=0.8)

# median line
fig.add_vline(
    x=median_duration,
    line_color='green',
    line_dash='dash',
    annotation_text='Median'
)

# mean line
fig.add_vline(
    x=mean_duration,
    line_color='red',
    line_dash='dash',
    annotation_text='Mean'
)

fig.show()

**Summary:**

The average session length is 11 minutes, while the median is only 5 minutes. This indicates that most sessions are short, but there are a few very long sessions (up to 711 minutes) that pull the average upward. The minimum duration is 0 minutes, which may correspond to incomplete or instantly closed sessions.

### 2.4. User Retention Rate

In [ ]:
#  Helper function: compute retention pivot and average retention
def compute_retention(visits: pd.DataFrame):
    """
    Compute weekly retention for user cohorts.
    
    Returns:
      retention_pivot (DataFrame): retention by cohort_week × lifetime_week
      avg_retention   (Series): average retention per lifetime_week
    """
    v = visits.copy()
    
    # First activity date for each user
    v['first_activity_date'] = v.groupby('uid')['start_ts'].transform('min')
    
    # Cohort week (first activity week) and activity week
    v['first_activity_week'] = v['first_activity_date'].dt.to_period('W').dt.start_time
    v['activity_week'] = v['start_ts'].dt.to_period('W').dt.start_time
    
    # Lifetime in weeks (how many weeks since the first activity)
    v['cohort_lifetime'] = (
        (v['activity_week'] - v['first_activity_week']) / np.timedelta64(1, 'W')
    ).astype(int)
    
    # Count active users per cohort_week × lifetime_week
    cohorts = (
        v.groupby(['first_activity_week', 'cohort_lifetime'])['uid']
        .nunique()
        .reset_index()
    )
    
    # Cohort size = number of users in week 0 for each cohort
    cohorts['cohort_users'] = cohorts.groupby('first_activity_week')['uid'].transform('first')
    
    # Retention rate = active users / cohort size
    cohorts['retention'] = cohorts['uid'] / cohorts['cohort_users']
    
    # Pivot: rows = cohort start week, columns = lifetime weeks
    retention_pivot = cohorts.pivot(
        index='first_activity_week',
        columns='cohort_lifetime',
        values='retention'
    ).round(3)
    
    # Average retention across cohorts for each lifetime week
    avg_retention = retention_pivot.mean(axis=0)
    
    return retention_pivot, avg_retention


# Compute retention tables
retention_pivot, avg_retention = compute_retention(visits)

# Show a sample: first 8 cohorts × first 12 weeks
display(retention_pivot.iloc[:8, :12])

In [ ]:
# Retention heatmap (first 12 cohorts × first 12 weeks)

# Make a copy and format index
heatmap_data = retention_pivot.copy()
heatmap_data.index = pd.to_datetime(heatmap_data.index).strftime('%Y-%m-%d')

# Limit to first 12 cohorts and first 12 lifetime weeks
heatmap_sample = heatmap_data.iloc[:12, :12]

# Plot heatmap
plt.figure(figsize=(13, 9))

ax = sns.heatmap(
    heatmap_sample,
    annot=True,  
    fmt='.0%',
    cmap='Blues',
    linewidths=0.5,
    linecolor='white',
    cbar_kws={'label': 'Retention rate'}
)

# Titles and labels
ax.set_title(
    'User Retention by Cohort and Lifetime Week\n(first 12 cohorts × first 12 weeks)',
    fontsize=16,
    pad=30
)
ax.set_xlabel('Cohort lifetime (weeks)', fontsize=12)
ax.set_ylabel('Cohort (first activity week)', fontsize=12)

# Show x-axis labels on top
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')

# Keep tick labels horizontal
plt.xticks(rotation=0)
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()

**📌 Note:** The table above shows only a sample (first 8 cohorts, 12 weeks). The full retention table is provided in the **Appendix**.

In [ ]:
# Average retention curve (first 12 weeks)

# Take first 12 lifetime weeks
avg_ret_12 = avg_retention.iloc[:12]

# Prepare a small DataFrame for plotting
avg_ret_df = (
    avg_ret_12
    .reset_index()
    .rename(columns={'cohort_lifetime': 'lifetime_week', 0: 'retention'})
)

# Line plot of average retention
fig = px.line(
    avg_ret_df,
    x='lifetime_week',
    y='retention',
    markers=True,
    title='Average User Retention (first 12 weeks)',
    labels={
        'lifetime_week': 'Cohort lifetime (weeks)',
        'retention': 'Retention rate'
    }
)

# Style the line (color)
fig.update_traces(line=dict(color='royalblue'))

fig.show()

**Summary**

**Average Retention Curve**

- The chart shows that retention drops steeply after the first week: from 100% in week 0 to under 10% in week 1.
- After that, retention stabilizes at a very low level (1–2%) and remains nearly flat over the following weeks.

**Cohort Heatmap**

- The heatmap illustrates the retention dynamics for different cohorts.
- The decline pattern is consistent across all cohorts: high churn after the first week, followed by stabilization at a low level.

No cohort demonstrates a significantly different behavior, indicating that the retention issue is structural rather than specific to a certain cohort.

**Conclusion:**
Overall, user retention is extremely low. Almost 90–95% of users drop off after the first week, and only about 1–2% remain active in the long term. This highlights the need for stronger onboarding and engagement strategies to increase user stickiness.

### 2.5. Session Length by Device

In [ ]:
# Calculate session duration stats by device
stats_by_device = (
    visits.groupby('device', observed=True)['session_duration_min']
          .agg(['count', 'mean', 'median'])
          .reset_index()
)

display(stats_by_device.round(1))

# Boxplot
fig = px.box(
    visits,
    x='device',
    y='session_duration_min',
    title='Session Length Distribution by Device',
    labels={
        'device': 'Device',
        'session_duration_min': 'Session length (minutes)'
    }
)

# Limit y-axis to remove extreme outliers
fig.update_yaxes(range=[0, 60])

# Style the box color
fig.update_traces(marker_color='royalblue')

fig.show()

**Summary:**  

Desktop sessions are longer: the median is about 6 minutes (IQR up to ~12–15), while touch sessions have a median of only 3 minutes.  

This means desktop users spend roughly twice as much time per visit, suggesting deeper engagement on desktop compared to mobile.

## Step 3. Exploring Sales Metrics

In this section, we will answer the following key questions:

- When do people start buying? (conversion delay between registration and first purchase)  
- How many orders do users place over time?  
- What is the average purchase size (AOV)?  
- How much money do users bring overall (LTV)?  
- How does the average purchase size differ by device?  

### 3.1. Conversion Delay Analysis

In [ ]:
# Build user-level table with first activity and first order dates
first_activity = (
    visits
    .groupby('uid')['start_ts']
    .min()
    .rename('first_activity_date')
)

first_order = (
    orders
    .groupby('uid')['buy_ts']
    .min()
    .rename('first_order_datetime')
)

users_conv = (
    first_activity
    .to_frame() 
    .merge(first_order, on='uid', how='left')
)

# Calculate conversion delay in days
users_conv['conversion_delay_days'] = (
    users_conv['first_order_datetime'] - users_conv['first_activity_date']
).dt.days

# Define buckets for conversion delay
bins = [-np.inf, -0.5, 0.5, 1.5, 3.5, 7.5, 30.5, 90.5, np.inf]
labels = ['No purchase', '0d', '1d', '2-3d', '4-7d', '8-30d', '31-90d', '90d+']

# Put each user into a conversion bucket
# If user has no purchase (NaN), we replace by -1 to fall into "No purchase"
users_conv['conversion_bucket'] = pd.cut(
    users_conv['conversion_delay_days'].fillna(-1),
    bins=bins,
    labels=labels,
    right=True
)

# Calculate distribution by buckets (counts and shares)
conv_counts = users_conv['conversion_bucket'].value_counts(sort=False)
conv_shares = (
    users_conv['conversion_bucket']
    .value_counts(sort=False, normalize=True) * 100
)

conv_df = (
    pd.DataFrame({
        'conversion_bucket': conv_counts.index,
        'count': conv_counts.values,
        'share': conv_shares.values
    })
    .reset_index(drop=True)
)

# Look at table: how many users in each bucket
display(conv_df[['conversion_bucket', 'count']])

# Plot conversion delay distribution
fig = px.bar(
    conv_df,
    x='conversion_bucket',
    y='share',
    title='Conversion Delay Distribution (%)',
    labels={
        'conversion_bucket': 'Conversion delay',
        'share': 'Share of users (%)'
    },
    text='share'
)

# Style bars and labels
fig.update_traces(
    texttemplate='%{text:.1f}%',
    marker_color='royalblue',
    hovertemplate='%{x}: %{y:.1f}%<extra></extra>'
)

fig.show()

**Summary**

Most users never make a purchase after registration — 84% fall into the "No purchase" category.
Among those who convert, the majority do so on the same day of registration (≈12%).
Only a very small share of users postpone their first purchase:

- 0.4–0.5% within 1–7 days
- 1% within 8–90 days
- 1.2% after 90+ days.

This indicates that conversion happens almost immediately if it happens at all. Long-term conversions are extremely rare, and the main focus should be on motivating users to purchase on day 0.

### 3.2. Number of Orders per User

**Helpers**

In [ ]:
# Count how many orders each user made
orders_per_user = (
    orders
    .groupby('uid')
    .size()
    .reset_index(name='orders_count')
)

# Calculate share of users by number of orders
order_dist = (
    orders_per_user['orders_count']
    .value_counts(normalize=True)
    .reset_index(name='share')
    .rename(columns={'index': 'orders_count'})
    .sort_values('orders_count')
)

# Keep only users with up to 10 orders (for readability)
max_orders = 10
dist_df = (
    order_dist[
        (order_dist['orders_count'] <= max_orders) & 
        (order_dist['share'] > 0)
    ]
    .reset_index(drop=True)
)

# Plot distribution (bar chart)
fig = px.bar(
    dist_df,
    x="orders_count",
    y="share",
    title="Distribution of Orders per User (up to 10 orders)",
    labels={
        "orders_count": "Number of orders",
        "share": "Share of users"
    },
    text="share"
)

# Style bars and labels
fig.update_xaxes(tickmode="linear", dtick=1)
fig.update_traces(
    marker_color="royalblue",
    texttemplate='%{text:.1%}',
    textposition="outside"
)
fig.update_yaxes(
    tickformat=".0%",
    range=[0, dist_df['share'].max() * 1.2]
)

fig.show()

**Summary:**

- The majority of users (≈83%) made only one order.
- About 12% of users made two orders, and the share drops sharply after that.
- Fewer than 1% of users placed more than five orders.
- There were isolated cases of users making over 100 orders, but their share is statistically insignificant (<0.1%) and was excluded from the chart for clarity.

Overall, the distribution of orders per user is highly skewed towards a single order, highlighting a low level of repeat purchases.

### 3.3. Average Purchase Size

In [ ]:
# Overall AOV for the whole period
# Formula: total revenue / total number of orders
total_revenue = orders['revenue'].sum()
total_orders = orders['revenue'].count()
aov = total_revenue / total_orders

print(f"Average Order Value (AOV): {aov:.2f}")

# AOV by month
# Сreate a copy and add month column
orders_monthly = orders.copy()
orders_monthly['month'] = orders_monthly['buy_ts'].dt.to_period('M')

# Сalculate AOV per month:
# sum(revenue) / number of orders in each month
aov_by_month = (
    orders_monthly
    .groupby('month')['revenue']
    .agg(['sum', 'count'])
    .reset_index()
)

aov_by_month['aov'] = (aov_by_month['sum'] / aov_by_month['count']).round(2)

# Сonvert Period ('month') → Timestamp (first day of the month)
aov_by_month['month'] = aov_by_month['month'].dt.to_timestamp()

# Оptional — drop incomplete last month
last_date = orders_monthly['buy_ts'].max()
end_of_last_month = last_date.to_period('M').to_timestamp(how='end')

# If the last month is incomplete, remove it from the chart
if last_date < end_of_last_month:
    last_full_month_start = (last_date.to_period('M') - 1).to_timestamp()
    aov_by_month = aov_by_month[aov_by_month['month'] <= last_full_month_start]

# Keep only the columns we need for plotting
aov_plot_df = aov_by_month[['month', 'aov']]

# Plot AOV by month
fig = px.line(
    aov_plot_df,
    x='month',
    y='aov',
    markers=True,
    title='Average Order Value (AOV) by Month',
    labels={
        'month': 'Month',
        'aov': 'Average Order Value'
    }
)

# Style line + hover
fig.update_traces(
    line=dict(width=3, color='royalblue'),
    marker=dict(size=6),
    hovertemplate='Month: %{x|%Y-%m}<br>AOV: %{y:.2f}<extra></extra>'
)

fig.update_xaxes(tickformat="%Y-%m")
fig.update_yaxes(tickformat=".2f")

fig.show()

**Summary**

- The analysis shows that the average order value (AOV) is about **5**.  
- When looking at the monthly trend, the AOV fluctuates between 4 and 6 units without a clear upward or downward pattern.  
- This suggests that the typical purchase size remained relatively stable during the observed period.

### 3.4. Customer Lifetime Value (LTV)

In [ ]:
# Add month of each order
orders['month'] = orders['buy_ts'].dt.to_period('M')

# Define cohort:
# First_order_month = first purchase month for each user
orders['first_order_month'] = (
    orders
    .groupby('uid')['month']
    .transform('min')
)

# Cohort lifetime in months:
# We convert Period to integer codes and subtract
orders['cohort_lifetime'] = (
    orders['month'].astype('int64') 
    - orders['first_order_month'].astype('int64')
)

# Cohort size = number of unique users in each first_order_month
cohort_users = (
    orders
    .groupby('first_order_month')['uid']
    .nunique()
)

# Total revenue per cohort and lifetime month
cohort_revenue = (
    orders
    .groupby(['first_order_month', 'cohort_lifetime'])['revenue']
    .sum()
    .reset_index()
)

# Add cohort size to each row
cohort_revenue['cohort_users'] = cohort_revenue['first_order_month'].map(cohort_users)

# LTV for each cohort & lifetime month:
# LTV_month = cohort_revenue / cohort_users
cohort_revenue['ltv'] = (
    cohort_revenue['revenue'] / cohort_revenue['cohort_users']
).round(2)

# Pivot table:
# Rows = cohorts (first_order_month), columns = lifetime months, values = LTV
ltv_pivot = cohort_revenue.pivot_table(
    index='first_order_month',
    columns='cohort_lifetime',
    values='ltv'
)

# Cumulative LTV across lifetime months
ltv_cum = ltv_pivot.cumsum(axis=1).round(2)

# Average cumulative LTV at month 6 across cohorts (sanity check)
if 6 in ltv_cum.columns:
    avg_ltv_6m = ltv_cum[6].dropna().mean()
    print(f"Average cumulative LTV at month 6 across cohorts: {avg_ltv_6m:.2f}\n")
else:
    print("Month 6 is not available in the dataset.\n")

# Show the cumulative LTV table
display(ltv_cum)

# Plot cumulative LTV heatmap

# Make a copy and format index
ltv_for_plot = ltv_cum.copy()
ltv_for_plot.index = (
    pd.to_datetime(ltv_for_plot.index.astype(str))
    .strftime('%Y-%m')
)

# Draw heatmap
plt.figure(figsize=(13, 9))
ax = sns.heatmap(
    ltv_for_plot,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    linewidths=0.5,
    linecolor='white',
    cbar_kws={'label': 'Cumulative LTV'}
)

# Titles and axis labels
ax.set_title(
    'Cumulative LTV by Cohort and Lifetime Month',
    fontsize=16,
    pad=30
)
ax.set_xlabel('Cohort lifetime (months)', fontsize=12)
ax.set_ylabel('Cohort (first order month)', fontsize=12)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')

plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

**Summary:**

- Cumulative LTV analysis shows that customers bring most of their value within the first 3–4 months of their lifetime.
- On average, a customer contributes 5–8 units in total, with the most valuable cohort (September 2017) reaching nearly 13 units. This means that the majority of revenue is generated shortly after acquisition, while long-term contribution is limited.

### 3.5. Average Order Value (AOV) by Device

In [ ]:
# Bring device info into the orders table (join by user ID)
# We use drop_duplicates() just in case the same user has many sessions.
orders_with_device = orders.merge(
    visits[['uid', 'device']].drop_duplicates(),
    on='uid',
    how='left'
)

# Calculate AOV by device:
# AOV_device = average order revenue per device
aov_by_device = (
    orders_with_device
    .groupby('device', observed=False)['revenue']
    .mean()
    .reset_index()
    .round(2)
)

display(aov_by_device)

# Prepare max value for x-axis
max_x = float(aov_by_device['revenue'].max()) * 1.2

# Plot horizontal bar chart
fig = px.bar(
    aov_by_device,
    x='revenue',
    y='device',
    orientation='h',
    title='Average Order Value (AOV) by Device',
    labels={'device': 'Device', 'revenue': 'AOV'},
    text='revenue'
)

fig.update_traces(
    texttemplate='%{text:.2f}',
    textposition='outside',
    marker_color='royalblue'
)

fig.update_xaxes(range=[0, max_x])
fig.show()

**Summary:**

- Average purchase size is higher on touch devices (≈10.6) compared to desktop (≈9.0).  
- This suggests that mobile users, although often less engaged per session, tend to make slightly larger purchases when they do order.  
- Desktop users generate more sessions overall, but their individual orders are smaller on average.


## Step 4. Marketing Metrics Analysis

- How much was spent? (overall, by source, and over time)  
- What was the customer acquisition cost (CAC)? (per source)  
- What was the return on investment (ROI) and payback period by cohorts?  
- How is marketing spend distributed by device?  

### 4.1. Analysis of Marketing Expenditures

In [ ]:
# Overall spend for the whole period
total_cost = int(costs['costs'].sum())
print(f"Overall marketing costs: {total_cost:,.0f}")

# Spend by source
# Group costs by source_id and sort descending
costs_by_sources = (
    costs.groupby('source_id', observed=False)['costs']
    .sum()
    .reset_index()
    .sort_values('costs', ascending=False)
)

# Convert source_id to string so it looks nicer on the chart
costs_by_sources['source_id'] = costs_by_sources['source_id'].astype(str)

# Plot total spend by source
fig = px.bar(
    costs_by_sources,
    x='source_id',
    y='costs',
    title='Marketing Costs by Source',
    labels={'source_id': 'Source', 'costs': 'Costs'},
    text='costs'
)

fig.update_traces(
    marker_color='royalblue',
    texttemplate='%{text:,.0f}',
    hovertemplate='Source %{x}<br>Costs: %{y:,.0f}<extra></extra>'
)

fig.update_yaxes(tickformat=',.0f')

fig.show()


# Spend by month (total)
# Create a month period column from date
costs['month'] = costs['dt'].dt.to_period('M')

# Group by month and sum costs
costs_by_month = (
    costs.groupby('month', observed=False)['costs']
    .sum()
    .reset_index()
)

# Convert Period to Timestamp 
costs_by_month['month'] = costs_by_month['month'].dt.to_timestamp()

# Plot total marketing costs over time
fig = px.bar(
    costs_by_month,
    x='month',
    y='costs',
    title='Marketing Costs Over Time',
    labels={'month': 'Month', 'costs': 'Costs'},
    text='costs'
)

fig.update_traces(
    marker_color='royalblue',
    texttemplate='%{text:,.0f}',
    hovertemplate='Month %{x|%Y-%m}<br>Costs: %{y:,.0f}<extra></extra>'
)

fig.update_yaxes(tickformat=',.0f')

fig.show()


# Spend by month and source
# Group by month and source_id
costs_msrc = (
    costs.groupby(['month', 'source_id'], observed=False)['costs']
    .sum()
    .reset_index()
)

# Convert month to timestamp and source_id to string
costs_msrc['month'] = costs_msrc['month'].dt.to_timestamp()
costs_msrc['source_id'] = costs_msrc['source_id'].astype(str)

# Plot stacked area chart: costs over time by source
fig = px.area(
    costs_msrc.sort_values('month'),
    x='month',
    y='costs',
    color='source_id',
    title='Monthly Marketing Costs by Source',
    labels={'month': 'Month', 'costs': 'Costs', 'source_id': 'Source'}
)

fig.update_yaxes(tickformat=',.0f')

fig.show()

**Summary:**

- The total marketing spend during the observed period amounted to 329,131.
- Most of the budget was allocated to a few major sources: Source 3 dominated with ≈141k (≈43% of total), followed by Source 4 (≈61k) and Source 5 (≈52k). Other sources accounted for much smaller shares, each contributing less than 15% of the total budget.
- Spending dynamics show a clear peak in November–December 2017 (≈37–38k per month) followed by a gradual decline in early 2018, indicating that the most intensive investment period was concentrated in late 2017.

### 4.2. Customer Acquisition Cost (CAC) Analysis

**Helpers**

In [ ]:
# First order date per user
first_order = (
    orders.groupby('uid', as_index=False)['buy_ts']
    .min()
    .rename(columns={'buy_ts': 'first_order_datetime'})
)

# Source of first purchase per user
users_sources = (
    first_order
    .merge(
        visits[['uid', 'start_ts', 'source_id']],
        on='uid',
        how='left'
    )
    .query('start_ts <= first_order_datetime')
    .sort_values(['uid', 'start_ts'])
    .drop_duplicates('uid')[['uid', 'source_id']]
)

# New customers per source (кол-во уникальных покупателей)
total_new_users_by_src = (
    users_sources.groupby('source_id', as_index=False)['uid']
    .nunique()
    .rename(columns={'uid': 'new_customers'})
)

# Total marketing spend per source
total_costs_by_src = (
    costs.groupby('source_id', as_index=False)['costs']
    .sum()
)

# CAC = total spend / number of new customers
cac_overall = total_costs_by_src.merge(
    total_new_users_by_src,
    on='source_id',
    how='left'
)
cac_overall['CAC'] = (cac_overall['costs'] / cac_overall['new_customers']).round(2)

# Prepare data for plotting
plot_df = cac_overall[['source_id', 'new_customers', 'CAC']].copy()
plot_df['source_id'] = plot_df['source_id'].astype(str)

# Plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart — New customers per source
fig.add_bar(
    x=plot_df['source_id'],
    y=plot_df['new_customers'],
    name='New Customers',
    marker_color='royalblue',
    text=plot_df['new_customers'],
    textposition='outside',
    hovertemplate='Source=%{x}<br>New customers=%{y:,}<extra></extra>'
)

# Line chart — CAC per source
fig.add_scatter(
    x=plot_df['source_id'],
    y=plot_df['CAC'],
    name='CAC',
    mode='lines+markers',
    line=dict(width=2, color='red'),
    marker=dict(size=7, color='red'),
    hovertemplate='Source=%{x}<br>CAC=%{y:.2f}<extra></extra>',
    secondary_y=True
)

# Layout settings
fig.update_layout(
    title='New Customers and CAC by Source',
    height=450,
    margin=dict(l=60, r=20, t=60, b=60),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5
    ),
    plot_bgcolor='white'
)

fig.show()

**Summary:**

- **Source 4** stands out as the most efficient channel, bringing in over 10k new customers with one of the lowest CAC values (~5.9).
- **Sources 9 and 10** also show very low CAC (~4–5), but their scale is limited.
- In contrast, **Source 3** attracted a similar number of customers as Source 4, yet with the highest CAC (~13.5), making it the least cost-effective option.
- **Sources 1, 2, and 5** delivered moderate results, with a reasonable balance of spend and customers but room for improvement.

**Recommendations:** 
- Focus on scaling Source 4, as it combines strong reach with low acquisition costs.
- Maintain Sources 9 and 10 due to their efficiency and explore whether they can be scaled.
- Reassess investments in Source 3 to optimize or reduce spending.
- Continue using Sources 1, 2, and 5, but test adjustments (targeting, segmentation, creatives) to improve CAC.

### 4.3. Marketing ROI Analysis

In [ ]:
# Prepare costs per cohort (use monthly costs as "cohort costs")
cohort_costs = costs_by_month.rename(columns={'month': 'first_order_month'})
cohort_costs['first_order_month'] = cohort_costs['first_order_month'].dt.to_period('M')

# Merge cohort revenue (ltv by cohort_lifetime) with monthly costs
report_roi = cohort_revenue.merge(
    cohort_costs,
    on='first_order_month',
    how='left'
)

# Number of users in each cohort (first_order_month)
cohort_users = (
    orders
    .groupby('first_order_month')['uid']
    .nunique()
)

# Add users column to report (cohort size)
report_roi['users'] = report_roi['first_order_month'].map(cohort_users)

# CAC per cohort (how much we spent per 1 user in cohort)
report_roi['cac'] = (report_roi['costs'] / report_roi['users']).round(2)

# Instant ROI per cohort–month
# ROI = LTV(month) / CAC
report_roi['roi'] = (report_roi['ltv'] / report_roi['cac']).round(2)

# Cumulative LTV and cumulative ROI over lifetime months
report_roi = report_roi.sort_values(['first_order_month', 'cohort_lifetime'])

report_roi['ltv_cum'] = (
    report_roi
    .groupby('first_order_month')['ltv']
    .cumsum()
)

report_roi['roi_cum'] = (report_roi['ltv_cum'] / report_roi['cac']).round(2)

# Prepare pivot table for heatmap (rows = cohorts, columns = lifetime months)
roi_cum_pivot = report_roi.pivot_table(
    index='first_order_month',
    columns='cohort_lifetime',
    values='roi_cum'
).round(2)

display(roi_cum_pivot)

# Plot heatmap of cumulative ROI by cohort
plt.figure(figsize=(13, 9))

ax = sns.heatmap(
    roi_cum_pivot,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    linewidths=0.5,
    linecolor='white',
    cbar_kws={'label': 'Cumulative ROI'}
)

ax.set_title('Cumulative ROI by Cohort and Lifetime Month', fontsize=16, pad=30)
ax.set_xlabel('Cohort lifetime (months)', fontsize=12)
ax.set_ylabel('Cohort (first order month)', fontsize=12)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')

plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

**Summary**

The ROI analysis shows how marketing investments paid off over time across customer cohorts:

- Most cohorts started with an ROI below 1.0 in the first months, meaning they were not yet profitable.
- Over time, ROI steadily increased as customer revenue accumulated.
- Several early cohorts (e.g., June–September 2017) reached or exceeded ROI > 1.0 within 6–9 months, showing that investments in those months became profitable in the medium term.
- Later cohorts (2018) generally demonstrate slower or weaker ROI growth, many staying below 1.0 within the observed period, which suggests longer payback times or less effective marketing spend.

### 4.4. CAC by device (Customer Acquisition Cost)

**Helpers**

In [ ]:
# First order date per user
first_order = (
    orders
    .groupby('uid', as_index=False)['buy_ts']
    .min()
    .rename(columns={'buy_ts': 'first_order_datetime'})
)

# First touch (first visit before first order): we take source and device
first_touch = (
    first_order
    .merge(
        visits[['uid', 'start_ts', 'source_id', 'device']],
        on='uid',
        how='left'
    )
    .query('start_ts <= first_order_datetime')
    .sort_values(['uid', 'start_ts'])
    .drop_duplicates('uid')[['uid', 'source_id', 'device']]
)

# Number of new customers by (source, device)
new_users_src_dev = (
    first_touch
    .groupby(['source_id', 'device'], observed=True)['uid']
    .nunique()
    .reset_index(name='new_customers')
)

# Total marketing costs per source
costs_by_src = (
    costs
    .groupby('source_id', as_index=False)['costs']
    .sum()
)

# Allocate costs between devices proportionally to new customers
alloc = new_users_src_dev.merge(costs_by_src, on='source_id', how='left')

# total new customers per source (for proportional split)
alloc['new_customers_src'] = alloc.groupby('source_id', observed=True)['new_customers'].transform('sum')

# allocated costs per (source, device)
alloc['allocated_costs'] = (
    alloc['costs'] * alloc['new_customers'] / alloc['new_customers_src']
).round(2)

# CAC per device
device_alloc = (
    alloc
    .groupby('device', observed=True, as_index=False)
    .agg(
        new_customers=('new_customers', 'sum'),
        allocated_costs=('allocated_costs', 'sum')
    )
)

device_alloc['cac_device'] = (
    device_alloc['allocated_costs'] / device_alloc['new_customers']
).round(2)

# LTV per device (average revenue per user on this device)
user_rev = (
    orders
    .groupby('uid', as_index=False)['revenue']
    .sum()
    .rename(columns={'revenue': 'user_revenue'})
)

user_dev_rev = user_rev.merge(first_touch[['uid', 'device']], on='uid', how='left')

ltv_by_device = (
    user_dev_rev
    .groupby('device', observed=True)['user_revenue']
    .mean()
    .reset_index(name='ltv_device')
    .round(2)
)

# ROI = LTV / CAC per device
roi_device = ltv_by_device.merge(device_alloc, on='device', how='left')
roi_device['roi_device'] = (
    roi_device['ltv_device'] / roi_device['cac_device']
).round(2)

# Final table for analysis
cols = ['device', 'ltv_device', 'cac_device',
        'new_customers', 'allocated_costs', 'roi_device']

display(roi_device[cols].reset_index(drop=True))

# Plot ROI by device (horizontal bar chart)
# auto-scale x-axis: 120% of max ROI, but not less than 1.0
max_x = max(1.0, float(roi_device['roi_device'].max()) * 1.2)

fig = px.bar(
    roi_device,
    x='roi_device',
    y='device',
    orientation='h',
    title='Average ROI by Device',
    labels={'device': 'Device', 'roi_device': 'ROI'},
    text='roi_device'
)

fig.update_traces(
    texttemplate='%{text:.2f}',
    textposition='outside',
    marker_color='royalblue'
)

fig.update_layout(
    xaxis=dict(range=[0, max_x]),
    height=400,
    margin=dict(l=80, r=20, t=60, b=40)
)

fig.show()

**Summary:**  

- Acquisition costs (CAC) are nearly the same for desktop and touch users (≈9).  
- However, desktop customers generate higher LTV (7.2 vs. 5.6), which leads to a stronger ROI (0.80 vs. 0.64).  
- This means that marketing investments in desktop users pay off faster, while touch campaigns remain less efficient within the observed period.

## Step 5. Conclusion and Recommendations

**Product usage.**  
Users generate most of their value during the first 3–4 months of activity. After this period, the marginal contribution drops significantly. This highlights the importance of effective onboarding and early engagement campaigns.  

**Sales performance.**  
On average, a customer contributes 5–8 revenue units in their lifetime, with the best cohort (Sep 2017) reaching nearly 13. Most revenue is therefore concentrated in the short-to-medium term, which defines the expected payback horizon.  

**Marketing efficiency.**  
- The total spend amounted to ~329k, peaking in Nov–Dec 2017.  
- **Source 4** was the strongest channel: it attracted >10k new users with one of the lowest CAC (~5.9).  
- **Sources 9 and 10** were highly efficient (CAC 4–5) but small in scale.  
- **Source 3** absorbed the largest share of the budget yet had the highest CAC (~13.5), making it the least efficient.  
- **Sources 1, 2, and 5** showed average performance with potential for optimization.  
- ROI analysis shows that only early cohorts (summer–fall 2017) achieved ROI > 1 within 6–9 months. Later cohorts (2018) performed weaker, with many not reaching payback during the observation window.  

**Recommendations.**  
1. **Scale Source 4** as the most effective channel (low CAC, large user base).  
2. **Maintain and explore growth for Sources 9 and 10**, given their efficiency, but assess scalability.  
3. **Reduce or optimize Source 3** to avoid overspending on an inefficient channel.  
4. **Keep Sources 1, 2, and 5**, but refine targeting and creatives to improve CAC and ROI.  
5. **Focus on retention efforts in the first 3–4 months**, where most of the user value is generated.  
6. Consider adjusting budget allocation after Dec 2017, as later cohorts showed weaker ROI.  

**Final note.**  
These recommendations are based on comparative analysis of LTV, CAC, ROI, and payback across cohorts and sources. They provide a clear path for reallocating the marketing budget toward higher-yield channels and strengthening early-stage retention.

## Appendix

**Retention table (% of users retained)**

**Note:**  
This table, presented in the appendix, refers to **Step 2.4. User Retention Rate**.  
It shows the percentage of users retained from each cohort by week.  
Values indicate what share of the original cohort remained active in subsequent weeks.  
For example, a value of 10% in week 1 means that only 10% of users who registered in that cohort returned the following week. 

In [ ]:
retention_pivot.index = pd.to_datetime(retention_pivot.index)
display(retention_pivot.style.format('{:.1%}'))